## Let's talk Keras

In [1]:
from keras.models import Sequential
model = Sequential()

In [2]:
from keras.layers import Dense, Dropout
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))

In [3]:
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

## Building a question classifier using neural networks

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
train_data = open('training_data.txt', 'r+')
test_data = open('test_dataset.txt', 'r+')
train = pd.DataFrame(train_data.readlines(), columns = ['Question'])
test = pd.DataFrame(test_data.readlines(), columns = ['Question'])

In [6]:
train.head()

,Question
0,DESC:manner How did serfdom develop in and the...
1,ENTY:cremat What films featured the character ...
2,DESC:manner How can I find a list of celebriti...
3,ENTY:animal What fowl grabs the spotlight afte...
4,ABBR:exp What is the full form of .com ?\n


In [7]:
train['QType'] = train.Question.apply(lambda x: x.split(' ', 1)[0])
train['Question'] = train.Question.apply(lambda x: x.split(' ', 1)[1])
train['QType-Coarse'] = train.QType.apply(lambda x: x.split(':')[0])
train['QType-Fine'] = train.QType.apply(lambda x: x.split(':')[1])
test['QType'] = test.Question.apply(lambda x: x.split(' ', 1)[0])
test['Question'] = test.Question.apply(lambda x: x.split(' ', 1)[1])
test['QType-Coarse'] = test.QType.apply(lambda x: x.split(':')[0])
test['QType-Fine'] = test.QType.apply(lambda x: x.split(':')[1])

In [8]:
train.head()

,Question,QType,QType-Coarse,QType-Fine
0,How did serfdom develop in and then leave Russ...,DESC:manner,DESC,manner
1,What films featured the character Popeye Doyle...,ENTY:cremat,ENTY,cremat
2,How can I find a list of celebrities ' real na...,DESC:manner,DESC,manner
3,What fowl grabs the spotlight after the Chines...,ENTY:animal,ENTY,animal
4,What is the full form of .com ?\n,ABBR:exp,ABBR,exp


In [9]:
train.pop('QType')
train.pop('QType-Fine')
test.pop('QType')
test.pop('QType-Fine')

0           dist
1           city
2           desc
3            def
4           date
         ...    
495          ind
496     currency
497        count
498    substance
499          def
Name: QType-Fine, Length: 500, dtype: object

In [10]:
classes = np.unique(np.array(train['QType-Coarse']))
classes

array(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype=object)

In [11]:
le = LabelEncoder()
le.fit(pd.Series(train['QType-Coarse'].tolist() + test['QType-Coarse'].tolist()).values)
train['QType-Coarse'] = le.transform(train['QType-Coarse'].values)
test['QType-Coarse'] = le.transform(test['QType-Coarse'].values)

In [12]:
def text_clean(corpus):
    cleaned_corpus = []
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus.append(' '.join(qs))
    return cleaned_corpus

In [13]:
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [14]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [15]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [16]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    if cleaning == True:
        corpus = text_clean(corpus)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]


    return corpus

In [17]:
all_corpus = pd.Series(train.Question.tolist() + test.Question.tolist()).astype(str)
all_corpus = preprocess(all_corpus, remove_stopwords = True)

In [18]:
train_corpus = all_corpus[0:train.shape[0]]
test_corpus = all_corpus[train.shape[0]:]

In [19]:
vectorizer = TfidfVectorizer()
tf_idf_matrix_train = vectorizer.fit_transform(train_corpus)
tf_idf_matrix_test = vectorizer.transform(test_corpus)

In [20]:
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Dropout, Input

In [21]:
y_train = keras.utils.to_categorical(train['QType-Coarse'], train['QType-Coarse'].nunique())
y_test = keras.utils.to_categorical(test['QType-Coarse'], train['QType-Coarse'].nunique())

In [22]:
model = Sequential()
model.add(Dense(128, activation='relu',
input_dim=tf_idf_matrix_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1027968   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
Total params: 1028742 (3.92 MB)
Trainable params: 1028742 (3.92 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
training_history = model.fit(tf_idf_matrix_train.toarray(), y_train, epochs=10, batch_size=100)

Epoch 1/10
55/55 [==============================] - 4s 36ms/step - loss: 1.7069 - categorical_accuracy: 0.4261
Epoch 2/10
55/55 [==============================] - 1s 19ms/step - loss: 1.3567 - categorical_accuracy: 0.7333
Epoch 3/10
55/55 [==============================] - 1s 19ms/step - loss: 0.8977 - categorical_accuracy: 0.8951
Epoch 4/10
55/55 [==============================] - 1s 19ms/step - loss: 0.5579 - categorical_accuracy: 0.9301
Epoch 5/10
55/55 [==============================] - 1s 19ms/step - loss: 0.3605 - categorical_accuracy: 0.9541
Epoch 6/10
55/55 [==============================] - 1s 19ms/step - loss: 0.2444 - categorical_accuracy: 0.9741
Epoch 7/10
55/55 [==============================] - 1s 19ms/step - loss: 0.1747 - categorical_accuracy: 0.9813
Epoch 8/10
55/55 [==============================] - 1s 19ms/step - loss: 0.1316 - categorical_accuracy: 0.9857
Epoch 9/10
55/55 [==============================] - 1s 27ms/step - loss: 0.1021 - categorical_accuracy: 0.9903
E

In [24]:
loss, accuracy = model.evaluate(tf_idf_matrix_test.toarray(), y_test,verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Testing Accuracy: 0.8540


In [25]:
import h5py
model_structure = model.to_json()
with open("question_classification_model.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("question_classification_weights.h5")